In [1]:
"""Vehicle Routing Problem"""
from __future__ import print_function
from six.moves import xrange
# or-tool CP solver
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
from math import sin, cos, sqrt, atan2, radians
import plotly
import googlemaps
import pandas as pd
import numpy as np

In [2]:
###########################
# Sample of Problem Data Definition #
###########################
class CityBlock():
    """City block definition"""
    @property
    def width(self):
        """Gets Block size West to East"""
        return 228/2

    @property
    def height(self):
        """Gets Block size North to South"""
        return 80

class DataProblem():
    """Stores the data for the problem"""
    def __init__(self):
        """Initializes the data for the problem"""
        self._num_vehicles = 5

        # Locations in block unit
        locations = \
                [(4, 4), # depot
                 (2, 0), (8, 0), # row 0
                 (0, 1), (1, 1),
                 (5, 2), (7, 2),
                 (3, 3), (6, 3),
                 (5, 5), (8, 5),
                 (1, 6), (2, 6),
                 (3, 7), (6, 7),
                 (0, 8), (7, 8)]
        # locations in meters using the city block dimension
        city_block = CityBlock()
        self._locations = [(
            loc[0]*city_block.width,
            loc[1]*city_block.height) for loc in locations]
        
        # Index of depot
        self._depot = 0

    @property
    def num_vehicles(self):
        """Gets number of vehicles"""
        return self._num_vehicles

    @property
    def locations(self):
        """Gets locations"""
        return self._locations

    @property
    def num_locations(self):
        """Gets number of locations"""
        return len(self.locations)

    @property
    def depot(self):
        """Gets depot location index"""
        return self._depot

In [3]:
###########################
# Example of a delivery problem
###########################
class DataProblem2():
    """Stores the data for the problem"""
    def __init__(self):
        """Initializes the data for the problem"""
        self._num_vehicles = 10

        # read nodes and depots
        df_nodes = pd.read_csv('data/deliveries.csv',sep='\s+')
        df_depots = pd.read_csv('data/stores.csv',sep='\s+')

        # get lat lon info as list
        nodes_latlon = df_nodes[['latitude', 'longitude']]
        depots_latlon = df_depots[['latitude', 'longitude']]

        # creat tuple list of lat lon
        locations = [tuple(x) for x in nodes_latlon.values]
        locations = [(tuple(depots_latlon.values[0]))]+locations

        # locations in meters using the city block dimension
        self._locations = [(loc[0],loc[1]) for loc in locations]
        
        # Index of depot
        self._depot = 0
        
        # Demand per location
        self._demands = [1]*len(locations)

    @property
    def num_vehicles(self):
        """Gets number of vehicles"""
        return self._num_vehicles

    @property
    def locations(self):
        """Gets locations"""
        return self._locations

    @property
    def num_locations(self):
        """Gets number of locations"""
        return len(self.locations)

    @property
    def depot(self):
        """Gets depot location index"""
        return self._depot
    
    @property
    def demands(self):
        """Get demand of locations"""
        return self._demands

In [4]:
#######################
# Problem Constraints #
#######################
def manhattan_distance(position_1, position_2):
    """Computes the Manhattan distance between two points"""
    return (abs(position_1[0] - position_2[0]) +
            abs(position_1[1] - position_2[1]))

def haversine_distance(position_1, position_2):
    """Computes the geographical distance between two points"""
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(position_1[0])
    lon1 = radians(position_1[1])
    lat2 = radians(position_2[0])
    lon2 = radians(position_2[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return(distance)

class CreateDistanceEvaluator(object): # pylint: disable=too-few-public-methods
    """Creates callback to return distance between points."""
    def __init__(self, data):
        """Initializes the distance matrix."""
        self._distances = {}

        # precompute distance between location to have distance callback in O(1)
        for from_node in xrange(data.num_locations):
            self._distances[from_node] = {}
            for to_node in xrange(data.num_locations):
                if from_node == to_node:
                    self._distances[from_node][to_node] = 0
                else:
                    self._distances[from_node][to_node] =(
                        haversine_distance(data.locations[from_node],
                                           data.locations[to_node]))
                                                           
    def distance_evaluator(self, from_node, to_node):
        """Returns the manhattan distance between the two nodes"""
        return self._distances[from_node][to_node]
    
    
class CreateDemandEvaluator(object): # pylint: disable=too-few-public-methods
    def __init__(self,data):
        """Initializes the demand list"""
        self._demands = data.demands
        
    def demand_evaluator(self, from_node, to_node):
        """Returns the demand for a node"""
        return self._demands[from_node]


def add_distance_dimension(routing, distance_evaluator):
    """Add Global Span constraint"""
    distance = "Distance"
    maximum_distance = 50
    routing.AddDimension(
        distance_evaluator,
        0, # null slack
        maximum_distance, # maximum distance per vehicle
        True, # start cumul to zero
        distance)
    # get the dimension by name
    distance_dimension = routing.GetDimensionOrDie(distance)
    # Try to minimize the max distance among vehicles.
    # /!\ It doesn't mean the standard deviation is minimized
    distance_dimension.SetGlobalSpanCostCoefficient(100)
    
def add_demand_dimension(routing, demand_evaluator):
    """Add a global capcity for each route"""
    capacity = "Capacity"
    routing.AddDimension(
        demand_evaluator,
        0,
        35,
        True,
        capacity)

In [5]:
###########
# Printer #
###########
class ConsolePrinter():
    """Print solution to console"""
    def __init__(self, data, routing, assignment):
        """Initializes the printer"""
        self._data = data
        self._routing = routing
        self._assignment = assignment
        self._stop = []
        self._route_num = []
        self._node_index = []

    @property
    def data(self):
        """Gets problem data"""
        return self._data

    @property
    def routing(self):
        """Gets routing model"""
        return self._routing

    @property
    def assignment(self):
        """Gets routing model"""
        return self._assignment
    @property
    def stop(self):
        return self._stop
    @property
    def route(self):
        return self._route_num
    
    @property
    def node(self):
        return self._node_index

    def print(self):
        """Prints assignment on console"""
        # Inspect solution.
        total_dist = 0

        for vehicle_id in xrange(self.data.num_vehicles):
            index = self.routing.Start(vehicle_id)
            plan_output = 'Route for vehicle {0}:\n'.format(vehicle_id)
            route_dist = 0
            route_load = 0
            route = []
            while not self.routing.IsEnd(index):
                node_index = self.routing.IndexToNode(index)
                next_node_index = self.routing.IndexToNode(
                    self.assignment.Value(self.routing.NextVar(index)))
                # Assign route number
                self._route_num.append(vehicle_id)
                # Assign stop number in a route
                self._stop.append(len(route))
                # Assign node index
                self._node_index.append(node_index)
                # Append node to route
                route.append(node_index)
                
                route_dist += haversine_distance(
                    self.data.locations[node_index],
                    self.data.locations[next_node_index])
                route_load += self.data.demands[node_index]
                
                plan_output += ' {node_index}({cuml_load}) -> '.format(
                    node_index=node_index, cuml_load=route_load)
                index = self.assignment.Value(self.routing.NextVar(index))

            node_index = self.routing.IndexToNode(index)
            total_dist += route_dist
            plan_output += ' {node_index}\n'.format(
                node_index=node_index)
            plan_output += 'Distance of the route {0}: {dist}\n'.format(
                vehicle_id,
                dist=route_dist)
            plan_output += 'Load of the route: {0}\n'.format(route_load)

            print(plan_output)
        print('Total Distance of all routes: {dist}'.format(dist=total_dist))

In [8]:
########
# Main #
########
def main():
    """Entry point of the program"""
    # Instantiate the data problem.
    data = DataProblem2()

    # Create Routing Model
    routing = pywrapcp.RoutingModel(data.num_locations, data.num_vehicles, data.depot)
    # Define weight of each edge
    distance_evaluator = CreateDistanceEvaluator(data).distance_evaluator
    demand_evaluator = CreateDemandEvaluator(data).demand_evaluator
    
    routing.SetArcCostEvaluatorOfAllVehicles(distance_evaluator)
    add_distance_dimension(routing, distance_evaluator)
    add_demand_dimension(routing, demand_evaluator)

    # Setting first solution heuristic (cheapest addition).
    search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)
    printer = ConsolePrinter(data, routing, assignment)
    printer.print()
    # Output in dataframe
    lat = [(data.locations[x])[0] for x in printer.node]
    lon = [(data.locations[x])[1] for x in printer.node]
    d = {'index':printer.node,'lat': lat, 'lon': lon,'route': printer.route, 'stop': printer.stop}

    df = pd.DataFrame(data=d)
    df.head()

    df.to_csv('./vrp_output.csv')

if __name__ == '__main__':
  main()

Route for vehicle 0:
 0(1) ->  190(2) ->  264(3) ->  277(4) ->  250(5) ->  241(6) ->  240(7) ->  274(8) ->  275(9) ->  12(10) ->  116(11) ->  6(12) ->  149(13) ->  290(14) ->  0
Distance of the route 0: 10.292126033856508
Load of the route: 14

Route for vehicle 1:
 0(1) ->  169(2) ->  76(3) ->  133(4) ->  38(5) ->  56(6) ->  208(7) ->  224(8) ->  52(9) ->  63(10) ->  85(11) ->  25(12) ->  155(13) ->  207(14) ->  143(15) ->  151(16) ->  148(17) ->  154(18) ->  119(19) ->  141(20) ->  134(21) ->  147(22) ->  106(23) ->  78(24) ->  10(25) ->  34(26) ->  5(27) ->  4(28) ->  7(29) ->  3(30) ->  1(31) ->  2(32) ->  21(33) ->  11(34) ->  8(35) ->  0
Distance of the route 1: 36.17527570081653
Load of the route: 35

Route for vehicle 2:
 0(1) ->  259(2) ->  301(3) ->  109(4) ->  99(5) ->  61(6) ->  53(7) ->  47(8) ->  104(9) ->  94(10) ->  77(11) ->  227(12) ->  110(13) ->  285(14) ->  191(15) ->  126(16) ->  67(17) ->  46(18) ->  44(19) ->  66(20) ->  269(21) ->  36(22) ->  213(23) ->  86(24)

In [14]:
data = DataProblem2()

# Create Routing Model
routing = pywrapcp.RoutingModel(data.num_locations, data.num_vehicles, data.depot)
# Define weight of each edge
distance_evaluator = CreateDistanceEvaluator(data).distance_evaluator
routing.SetArcCostEvaluatorOfAllVehicles(distance_evaluator)
add_distance_dimension(routing, distance_evaluator)

# Setting first solution heuristic (cheapest addition).
search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
# Solve the problem.
assignment = routing.SolveWithParameters(search_parameters)
printer = ConsolePrinter(data, routing, assignment)
printer.print()



Route for vehicle 0:
 0 ->  93 ->  132 ->  190 ->  129 ->  184 ->  170 ->  130 ->  173 ->  236 ->  233 ->  205 ->  139 ->  115 ->  103 ->  180 ->  90 ->  102 ->  283 ->  293 ->  246 ->  165 ->  112 ->  113 ->  253 ->  108 ->  254 ->  297 ->  75 ->  23 ->  28 ->  192 ->  277 ->  264 ->  265 ->  299 ->  247 ->  237 ->  268 ->  214 ->  197 ->  229 ->  125 ->  64 ->  117 ->  107 ->  54 ->  101 ->  30 ->  31 ->  166 ->  199 ->  202 ->  188 ->  159 ->  12 ->  1 ->  210 ->  13 ->  280 ->  232 ->  226 ->  234 ->  243 ->  223 ->  235 ->  275 ->  274 ->  240 ->  17 ->  45 ->  11 ->  8 ->  72 ->  0
Distance of the route 0: 56.00940634137887

Route for vehicle 1:
 0 ->  65 ->  0
Distance of the route 1: 1.2256153417409992

Route for vehicle 2:
 0 ->  96 ->  221 ->  289 ->  266 ->  56 ->  38 ->  133 ->  295 ->  194 ->  89 ->  244 ->  271 ->  276 ->  58 ->  169 ->  97 ->  0
Distance of the route 2: 20.29648799269516

Route for vehicle 3:
 0 ->  259 ->  204 ->  95 ->  281 ->  267 ->  230 ->  224 ->  